In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import cudf
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.logreg.logreg_cv_trainer as cv
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [ ]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = cudf.read_parquet("../artifacts/features/base/tr_df1.parquet")
test_df1 = cudf.read_parquet("../artifacts/features/base/test_df1.parquet")

In [ ]:
# Check tuning results
study_name = "logreg_v1"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()

In [ ]:
# Create OOF and test predictions
importlib.reload(cv)
params = {
    "C": 0.2131225478825375
}
trainer = cv.LogRegCVTrainer(params=params)
oof, test_preds = trainer.fit(tr_df1, test_df1)
np.save("../artifacts/preds/base/oof_single_1.npy", oof)
np.save("../artifacts/preds/base/test_single_1.npy", test_preds)

In [ ]:
te.send_telegram_message("LogReg Training Complete!")